In [1]:
!python -V

Python 3.9.12


In [2]:
import pandas as pd

In [3]:
import pickle 

In [4]:
import seaborn as sns
import matplotlib.pyplot as plt

In [5]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge


from sklearn.metrics import mean_squared_error

In [5]:
df = pd.read_parquet("../notebooks/data/green_tripdata_2021-01.parquet")

# #### Next:
# we will calculate the "duration" in minutes, and plot the distribution of it.

df['duration']= df.lpep_dropoff_datetime - df.lpep_pickup_datetime
df.duration = df.duration.apply(lambda td: td.total_seconds()/60)

df = df[(df.duration >= 1) & (df.duration <= 60)]

categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']

df[categorical]=df[categorical].astype(str)

#### Next:
Now we are going to train a model. To train a model, we need to do something with the categorical variables. Our categorical var's types are int,  and since we are going to do "one-hot encoding", we can be using dictionary vectorizer, and it treats everything that is not a number as categorical var, however here our cat vars are int, so it will think they are numerical vars, so we will convert them to string.

In [ ]:
train_dicts = df[categorical + numerical].to_dict(orient='records')

dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

target= 'duration'
y_train = df[target].values

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

mean_squared_error(y_train, y_pred, squared=False)

In [ ]:
sns.distplot(y_pred, label='prediction')
sns.distplot(y_train, label='actual')

plt.legend()

## Note:
Here we can see the predicted values are pretty different than actual values, which is why we didn't expect this model to particularly be good.

#### Next:
    for calculating the performance of this model on the training dataset, we will use mean_square_errors from sikitlearn

In [1]:
def read_datarfame(filename):
    df = pd.read_parquet(filename)

    # #### Next:
    # we will calculate the "duration" in minutes, and plot the distribution of it.

    df['duration']= df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds()/60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']

    df[categorical]=df[categorical].astype(str)
    
    return df

In [6]:
df_train = read_datarfame('../notebooks/data/green_tripdata_2021-01.parquet')
df_val = read_datarfame('../notebooks/data/green_tripdata_2021-02.parquet')

In [7]:
len(df_train), len(df_val)

(73908, 61921)

In [18]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' +df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' +df_val['DOLocationID']

In [20]:
categorical = ['PU_DO'] # ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)



In [9]:
target= 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [26]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

7.4793657829373155

In [29]:
with open('../notebooks/models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv,lr), f_out)

In [ ]:
### Next:
Let's try Lasso regression

In [24]:
lr = Lasso(alpha=0.0001)
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

7.616617761096093

In [23]:
lr = Ridge(alpha=0.01)
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

11.34256591918933